# Preceptor roster automation.

In [3]:
# ==============================
# Excel Scheduling Automation
# ==============================

# Import statements
import pandas as pd
from datetime import datetime, timedelta, date
from openpyxl import load_workbook

## TODO : 

* Add Remarkds column (empty) besides the Full name column.
* Add Auto fit of clumns especially the date and name part.
* Mark rows / text for with options given as "second sunday" with green background or with some cell color for easy recognition.
* For evening ranking, remove all those names which has morning preference in ["M", "SSM"] and morning count is zero.
* For evening ranking, remove all those names which has noon preference in ["N", "SSN"] and noon count is zero. Let us first consider them in morning or noon before scheduling for evening.
* Should skip dropping columns for Morning, noon, evening tabs. Lets keep them all same layout
* Can update correct morning ranks in other two tabs, same for noon rank and evening rank.

In [ ]:
# ==============================================================
# Step 1: Define function to read input Excel file
# ==============================================================

def process_excel_schedule(file_path, names_per_slot=4, update_file=False, consolidate_in_single_tab = False):
    """
    Function to process scheduling data from input Excel file.
    Args:
        file_path (str): Path to input Excel file.
        names_per_slot (int): Number of rows to export to each slot tab (default=4).
    """

    # Read the Excel file with headers on the 2nd row (index 1)
    df = pd.read_excel(file_path, sheet_name=0, header=1)

    # ==============================================================
    # Step 2: Select and rename relevant columns
    # ==============================================================

    columns_map = {
        "Date (L)": "last date",
        "Full Name": "full name",
        "1st": "morning count",
        "2nd": "noon count",
        "3rd": "evening count",
        "Total": "total count",
        "M": "morning date",
        "M-Rank": "morning rank",
        "N": "noon date",
        "N-Rank": "noon rank",
        "E": "evening date",
        "E-Rank": "evening rank",
        "Morning": "morning preference",
        "Mid-Noon": "noon preference",
        "Evening": "evening prefrence"
    }

    df = df[list(columns_map.keys())]
    df = df.rename(columns=columns_map)
    df["last date"] = df["last date"].fillna("2000-01-01")

    # Preparatory part constant irrespective of data.
    today = date.today()
    cuttoff_datetime =(today - timedelta(days=5*30))
    #    morning["morning date"] = pd.to_datetime(morning["morning date"], errors='coerce').dt.date
    cutoff_date = date(cuttoff_datetime.year, cuttoff_datetime.month, cuttoff_datetime.day)  # Approx 5 months

    df["last date"] = pd.to_datetime(df["last date"], errors= "coerce").dt.date

    # df.drop("last date", axis=1, inplace=True)

    print(f"\n original df row count ${len(df)}")
    # ==============================================================
    # For the 1st slot (morning) Scheduling
    # ==============================================================

    # Step 5: Copy dataframe
    morning = df.copy()

    # Step 6: Keep only rows where preference is "M" or "SSM"
    morning = morning[morning["morning preference"].isin(["M", "SSM"])]


    print(f"\n Morning Schedule dataframe row count step 6 : {len(morning)}")
    morning["morning date"] = pd.to_datetime(morning["morning date"], errors='coerce').dt.date
    morning = morning[morning["morning date"] < cutoff_date]

    # Remove all those who have conducted satsang in noon in last 5 months.
    morning["noon date"] = pd.to_datetime(morning["noon date"], errors='coerce').dt.date
    morning_part1 =  morning[morning["noon date"] < cutoff_date]
    morning_part2 = morning[morning["noon date"] > today]
    morning = pd.concat([morning_part1, morning_part2], ignore_index=True)

    # Step 8: Sort ascending by morning date
    morning = morning.sort_values(by=['morning date','noon date', 'evening date'], ascending=True)

    # Extra clean up of columns that are not relavant for morning slot
    morning.drop(columns=["noon date", "noon rank", "evening date", "evening rank", "noon preference", "evening prefrence"], axis=1, inplace=True)


    # Step 9: Populate morning rank 1..n
    morning["morning rank"] = range(1, len(morning) + 1)

    print(f"\n Morning Schedule dataframe row count step 9 : {len(morning)}, names per slot : {names_per_slot}")
    morning_top = morning.head(names_per_slot)
    
    # Print top 3 rows (limited columns)
    # print("\nTop few Morning Schedule:")
    # print(morning)

    # ==============================================================
    # For the 2nd slot (noon) Scheduling
    # ==============================================================

    # Step 12: Copy original dataframe
    noon = df.copy()

    # Step 13: Remove rows already exported in morning tab
    exported_names = set(morning_top["full name"].unique())
    noon = noon[~noon["full name"].isin(exported_names)]

    # Step 14: Keep only where noon preference is "N" or "SSN"
    noon = noon[noon["noon preference"].isin(["N", "SSN"])]

    print(f"\n noon Schedule dataframe row count : {len(noon)}")

    # Step 15: Remove rows where morning date is within last 5 months
    noon["morning date"] = pd.to_datetime(noon["morning date"], errors='coerce').dt.date
    removed_noon = noon[noon["morning date"] >= cutoff_date]
    removed_noon = removed_noon[removed_noon["morning date"] <= today]
    noon_part1 =  noon[noon["morning date"] < cutoff_date]
    noon_part2 = noon[noon["morning date"] > today]
    noon = pd.concat([noon_part1, noon_part2], ignore_index=True)
    print(f"\n noon Schedule dataframe row count after removing last 5 months data for morning : {len(noon)}")
    print(f"\n removed names: count = {len(removed_noon)}\n {removed_noon["full name"]}")

    # Step 16: Sort by noon date ascending
    noon["noon date"] = pd.to_datetime(noon["noon date"], errors='coerce').dt.date
    noon = noon.sort_values(by=['noon date', 'morning date', 'evening date'], ascending=True)
    
    # extra clean up for removing columns not relavant for noon.
    noon.drop(columns= ["evening date","evening rank", "morning preference", "evening prefrence"], axis=1, inplace=True)

    # Step 17: Update noon rank
    noon["noon rank"] = range(1, len(noon) + 1)


    # Step 18: Output top `names_per_slot` rows to "noon" tab
    noon_top = noon.head(names_per_slot)

    # Step 19: Print top 3 rows
    print("\nTop 5 Noon Schedule:")
    print(noon[["full name", "noon count", "noon date", "noon rank", "noon preference"]].head(5))

    # print("\nJagdish ji Morning rank:")
    # print(morning[morning["full name"].str.contains("Jagdish")])
    # print("\nJagdish ji Afternoon rank:")
    # print(noon[noon["full name"].str.contains("Jagdish")])

    # Step 20: create a dataframe for evening
    evening = df.copy()

    # Step 21 : remove all rows which has values other than "SSE", "E" in evening preference.
    evening = evening[evening["evening prefrence"].isin(["E","SSE"])]
    print(f"\n step 21 evening Schedule dataframe row count : {len(evening)}")

    # Step 22 : remove all rows that are either already considered in morning or afternoon slots.
    exported_morning_names = set(morning_top["full name"].unique())
    evening = evening[~evening["full name"].isin(exported_morning_names)]
    exported_noon_names = set(noon_top["full name"].unique())
    evening = evening[~evening["full name"].isin(exported_noon_names)]
    print(f"\n step 22 evening Schedule dataframe row count : {len(evening)}")
    
    # Step 23 : remove all those whose dates in any of the slot is less then 3 months.
    cutoff_date = today - timedelta(days=3*30)  # Approx 3 months
    evening["morning date"] = pd.to_datetime(evening["morning date"], errors='coerce').dt.date
    removed_eve1 =  evening[evening["morning date"] > cutoff_date]
    removed_eve1 =  removed_eve1[evening["morning date"] < today]
    eve_part1 =  evening[evening["morning date"] < cutoff_date]
    eve_part2 = evening[evening["morning date"] > today]
    evening = pd.concat([eve_part1, eve_part2], ignore_index=True)

    evening["noon date"] = pd.to_datetime(evening["noon date"], errors='coerce').dt.date
    removed_eve2 =  evening[evening["noon date"] > cutoff_date]
    removed_eve2 =  removed_eve2[evening["noon date"] < today]
    eve_part3 = evening[evening["noon date"] < cutoff_date]
    eve_part4 = evening[evening["noon date"] > today]
    evening = pd.concat([eve_part3, eve_part4], ignore_index=True)
    
    print(f"\n step 22 evening Schedule dataframe row count : {len(evening)}") 
    print(f"\n removed names considering morning date: count = {len(removed_eve1)}\n {removed_eve1["full name"]}") 
    print(f"\n removed names considering noon date: count = {len(removed_eve2)}\n {removed_eve2["full name"]}") 
    
    # Step 16: Sort by noon date ascending
    evening["evening date"] = pd.to_datetime(evening["evening date"], errors='coerce').dt.date
    evening = evening.sort_values(by=['evening date', 'noon date', 'morning date'], ascending=True)
    
    # Step 17: Update noon rank
    evening["evening rank"] = range(1, len(evening) + 1)


    # Step 18: Output top `names_per_slot` rows to "noon" tab
    evening_top = evening

    # Step 19: Print top 3 rows
    # print("\nEvening Schedule:")
    # print(evening)
    
    if update_file:
        print('updating file')
        # Create Excel writer (openpyxl mode to preserve existing data)
        with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            # ==============================================================
            # Step 3: Create 3 tabs in the same file
            # ==============================================================
            for sheet_name in ["morning", "noon", "evening"]:
                pd.DataFrame().to_excel(writer, sheet_name=sheet_name, index=False)

            if consolidate_in_single_tab:
                consolidated_df = pd.concat([morning_top, noon_top, evening_top])
                consolidated_df.to_excel(writer, sheet_name="morning", index=False)
            else:
                # Step 10: Output top `names_per_slot` rows to "morning" tab
                morning_top.to_excel(writer, sheet_name="morning", index=False)

                # Step : output top `names_per_slot` rows to noon tab.
                noon_top.to_excel(writer, sheet_name="noon", index=False)

                # Step : output all rows to evening tab
                evening_top.to_excel(writer, sheet_name="evening", index=False)

        print("\n✅ Excel file updated successfully with morning and noon schedules.")

    return

def autofit_column_width(file_path, sheet_name):
    # Load the workbook
    wb = load_workbook(file_path)
    sheet = wb[sheet_name]  # or wb['SheetName'] for a specific sheet
    
    # Iterate through columns and adjust width
    for col in sheet.columns:
        max_length = 0
        column = col[0].column_letter  # Get the column letter (e.g., 'A', 'B')
        for cell in col:
            try:  # Handle potential non-string values
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except TypeError:
                pass
        adjusted_width = (max_length + 2)  # Add a little padding
        sheet.column_dimensions[column].width = adjusted_width

input_file_path = "c:\\Users\\Ashish\\Downloads\\Sunday_date-wise satsang record.xlsx"
update_file = True
consolidate_in_single_tab = True
process_excel_schedule(input_file_path, 12, update_file, consolidate_in_single_tab)
if update_file:
    autofit_column_width(input_file_path, "morning")
    autofit_column_width(input_file_path, "noon")
    autofit_column_width(input_file_path, "evening")


 original df row count $226

 Morning Schedule dataframe row count step 6 : 206

 Morning Schedule dataframe row count step 9 : 169, names per slot : 32

 noon Schedule dataframe row count : 149

 noon Schedule dataframe row count after removing last 5 months data for morning : 133

 removed names: count = 16
 3       Shri Subodh Ganpat Rahate
4              Smt. Indira Prasad
5        Miss. Triveni S. Khedkar
6               Smt. Anupama paul
7          Smt. Preeti Maheshwari
8      Shri Nikhil Vinod Wankhede
9               Shri Sunil Butani
39          Smt. Mangala Moralwar
42             Smt. Usha Rathinam
52               Shri Vinit Gupta
70           Shri Krishnanad Bhat
122                Smt. Anju Dhar
132            Smt. Uma Bhargava 
133         Smt. Yogita Hatangadi
144        Smt. Madhura Vyawahare
154            Shri. Mahesh Kakoo
Name: full name, dtype: object

Top 5 Noon Schedule:
                    full name  noon count   noon date  noon rank  \
0  Shri Haresh Damjee 

C:\Users\Ashish\AppData\Local\Temp\ipykernel_14176\2917035650.py:158: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  removed_eve1 =  removed_eve1[evening["morning date"] < today]
C:\Users\Ashish\AppData\Local\Temp\ipykernel_14176\2917035650.py:165: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  removed_eve2 =  removed_eve2[evening["noon date"] < today]



✅ Excel file updated successfully with morning and noon schedules.
